## torch.nn.Module AND torch.nn.Parameter
##### 간단하게 두 개의 선형 레이어가 있는 간단한 모델을 만들어보자. 그리고 활성화 함수, 해당 인스턴스를 확인하고 매개변수를 보자.

In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()
        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0374,  0.0263,  0.0571,  ..., -0.0867, -0.0959, -0.0104],
        [-0.0582, -0.0152, -0.0739,  ..., -0.0918, -0.0167,  0.0212],
        [-0.0754, -0.0262, -0.0191,  ...,  0.0883, -0.0620,  0.0323],
        ...,
        [-0.0636, -0.0395, -0.0972,  ..., -0.0005, -0.0262, -0.0414],
        [ 0.0947,  0.0294, -0.0543,  ...,  0.0540, -0.0505,  0.0392],
        [ 0.0173,  0.0258,  0.0978,  ...,  0.0816, -0.0123,  0.0325]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0818,  0.0157, -0.0359,  0.0848, -0.0720,  0.0015,  0.0391, -0.0669,
        -0.0760,  0.0280,  0.0111,  0.0127, -0.0300, -0.0623, -0.0375, -0.0961,
         0.0116,  0.0589, -0.0826, -0.09

## 공통 레이어 유형
### 선형 레이어
##### 가장 기본적인 유형의 신경망 계층은 linear 또는 fully connected 레이어이다. 모든 입력이 모든 요소에 영향을 미치는 레이어이다.
##### 만약 모델에 m 입력과 n 출력이 있는 경우 가중치는 m x n 행렬이다.

In [3]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1,3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
  print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.3844, 0.9330, 0.1955]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.0963,  0.4044,  0.4871],
        [ 0.1712,  0.4085, -0.4989]], requires_grad=True)
Parameter containing:
tensor([ 0.0112, -0.4379], requires_grad=True)


Output:
tensor([[ 0.4467, -0.0884]], grad_fn=<AddmmBackward0>)


### 컨벌루션 레이어
##### conv layer는 높은 수준의 데이터를 처리하도록 구축되었다. (ex. 공간, nlp .. )

In [5]:
import torch.functional as F

class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### 반복 레이어
##### 순환 신경망은 순차 데이터에 사용된다. 숨겨진 상태를 유지하여 해당 내용에 대한 일종의 메모리 역할을 한다.
##### RNN 레이어의 내부 구조 또는 그 변형인 LSTM을 테스트 해보자. -> LSTM 기반 품사 태거

In [6]:
class LSTMTagger(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

## 기타 레이어
### 데이터 조작 계층
##### 학습 과정에는 참여하지 않지만 데이터(텐서)를 조작하는 레이어이다.

In [11]:
my_tensor = torch.rand(1,6,6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

# size가 pooling layer의 사이즈 만큼 줄어든다.
out = maxpool_layer(my_tensor)
print(out.shape)

tensor([[[0.0447, 0.2731, 0.7717, 0.5675, 0.0985, 0.9274],
         [0.7809, 0.8720, 0.1859, 0.6821, 0.7895, 0.8516],
         [0.8636, 0.1887, 0.6128, 0.4421, 0.4876, 0.4232],
         [0.1133, 0.7171, 0.5039, 0.9983, 0.2850, 0.6433],
         [0.6282, 0.8662, 0.5032, 0.5585, 0.5198, 0.9074],
         [0.4858, 0.9407, 0.5888, 0.7439, 0.4679, 0.2335]]])
tensor([[[0.8720, 0.9274],
         [0.9407, 0.9983]]])
torch.Size([1, 2, 2])


##### 각 사분면의 최대 값이다. (6 x 6)

##### 정규화 레이어 한 레이어의 출력을 다시 중앙에 배치하고 정규화를 수행한다. 그러면 경사도 폭발/소멸 없이 학습률을 높일 수 있다.

In [8]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)
print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[14.7523,  8.6027, 16.8663, 19.6900],
         [13.0565, 12.4996, 13.8683, 21.3891],
         [ 8.6157, 10.2775, 15.0089, 19.1571],
         [16.9850, 10.0303, 16.9798, 14.7375]]])
tensor(14.5323)
tensor([[[-0.0553, -1.5640,  0.4633,  1.1560],
         [-0.5956, -0.7501, -0.3704,  1.7162],
         [-1.1251, -0.7229,  0.4221,  1.4260],
         [ 0.8110, -1.6393,  0.8091,  0.0191]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(0., grad_fn=<MeanBackward0>)


##### Dropout 레이어는 희소 표현을 장려한다. 즉, 축소된 데이터 세트에 대해 학습한다.

In [9]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 1.4565, 0.4831, 0.8894],
         [1.5687, 1.1433, 0.5709, 0.0000],
         [0.5527, 0.7883, 0.0000, 0.2089],
         [0.0310, 1.6056, 0.0036, 0.9695]]])
tensor([[[1.5673, 1.4565, 0.0000, 0.0000],
         [1.5687, 0.0000, 0.5709, 1.6012],
         [0.0000, 0.0000, 0.0000, 0.2089],
         [0.0310, 1.6056, 0.0000, 0.0000]]])


### activation function
##### 이전 층의 결과값을 변환하여 다른 층의 뉴런으로 신호를 전달하는 역할을 한다. 모델의 복잡도를 올려 비선형 문제를 해결하는데 중요한 역할을 한다.
##### 단순 은닉층을 많이 쌓는다고 해서 비선형 문제를 해결할 순 없다. 활성화 함수를 사용하여 출력값이 linear하게 나오지 않게 하여 선형 분류기를 비선형 시스템으로 만든다.
##### 종류로는 sigmoid, ReLU, tanh 등이 있다.

### loss function
##### 모델이 예측이 실제 값과 얼마나 떨어져 있는지 알 수 있다.
##### 종류로는 mse, crossentropy 등이 있다.